# Key Exchange

Being able to exchange information securely is a foundation of how we communicate in this day and age. Whenever things communicate electronically, whether that be two people sending emails back and forth or devices syncing on a network, these communications need to be kept private and secure. That is where the cryptographic process of key exchange comes into play. 

To begin, we'll import some libraries that will be useful in our explanation:

In [ ]:
import base64
from hashlib import shake_256
import hmac, hashlib, secrets
from helper_methods import *
import secrets

Next, we will dive into the backbone of sharing information securely: the Diffie-Hellman Algorithm. 

## The Basics of Diffie-Hellman

Before diving into all of the potential subtleties and problems that will arise with this algorithm, let's look at a basic example, with reasonable numbers. 

Imagine two people, Alice and Bob, who need to communicate something securely over an insecure channel. They need a way to share a secret number without anyone else figuring it out. Here's how they can do it:

First, they both agree on a public base, or generator, $g$, and a prime number, $q$. For this example, we'll choose $g=2$ and $q=11$. They'll also each choose a secret integer. Alice chooses $a=3$, and Bob will choose $b=4$. 

Now that they have these numbers, they each compute a public value, using this formula: $$ P=g^{secret} \pmod q $$
For Alice, this looks like: $$ p_A=2^3 = 8\pmod {11}$$
So Alice's public key is 8.
For Bob, this looks ike: $$ p_B=2^4 = 5 \pmod {11} $$
So Bob's public key is 5. They now both share their public values to the world. 

Each person now uses their own secret integer and the other person's public value to compute this shared secret. Alice computes: $$ k = (p_B)^a = 5^3 \pmod {11}$$ $$ k = 125 = 4 \pmod {11}$$
Thus, Alice gets her $k=4$. 

Bob does the same: $$ k=(p_A)^b = 8^4 = 4096 = 4 \pmod {11}$$
You can confirm both of these results by running the following code blocks:

In [2]:
(5**3) % 11

4

In [3]:
(8**4) % 11

4

We can see that they both get the same secret key, despite not communicating it. This is the essence of the Diffie-Hellman algorithm, and the ability to share a secret without anyone else knowing it is the backbone of sending messages securely. 

### Generalizing Diffie-Hellman
Above was just a simple example to show that this algorithm works. We will now go over this algorithm in general, to give a better idea of how it works. 

In general, we will use a value of $q$ that is massive as our modulo. Making $q$ very large helps defend against brute force attacks. Nowadays, the minimun value for $q$ that is considered safe is a $q$ that is 2048 bits long in binary, or on the order of $2^{2048}$. To put that in context, run this code block: 

In [4]:
q = 2**2048
q

32317006071311007300714876688669951960444102669715484032130345427524655138867890893197201411522913463688717960921898019494119559150490921095088152386448283120630877367300996091750197750389652106796057638384067568276792218642619756161838094338476170470581645852036305042887575891541065808607552399123930385521914333389668342420684974786564569494856176035326322058077805659331026192708460314150258592864177116725943603718461857357598351152301645904403697613233287231227125684710820209725157101726931323469678542580656697935045997268352998638215525166389437335543602135433229604645318478604952148193555853611059596230656

To get an idea of the scale of that number, we can print it in scientific notation:

In [5]:
print_large_int_sci(q)

'3.2317e+616'

For reference, the total number of atoms in our entire universe is said to be about $10^{80}$. Luckily for us, modern software is optimized to compute modular arithmetic, and using this modulo makes sure that our numbers never get too large. We use functions like python's built-in `pow` to compute modular exponentiation. 

Another key requirement is to pick a value for $q$ that is a **safe prime**. 

-----------------------------------------------------------------

#### Safe Primes
In short, a safe prime is a prime number $p$ such that: $$ p = 2p_0 + 1$$ where $p_0$ is also a prime number. Having our $q$ be a safe prime is essential in maintaining the security of the algorithm, as it helps us avoid small subgroup attacks. Small subgroup attacks essentially find small "minicycles" within your chosen modulo operator and use them to obtain part or all of your secret key. 

For now, though, it's enough to understand that not any prime can be a suitable value for $q$, and this specific $q$ is used because it is a safe prime, and because it is a massive number, both of which help to deter such an attack. 

-----------------------------------------------------------------

Moving onto the other constant we will use, $g$ also has a few rules for picking it. In most cases, $g$ is picked such that as it is raised to successive powers mod $q$, it takes on many unique values before repeating. For safe primes, you can generally use $g=2$ as your generator, although you must make sure that $q$ is a safe prime, as we mentioned above, and make sure that: $$ g^{(q-1)/2} \neq 1 \pmod q $$ This just makes sure that $g$ doesn't lie in a smaller subgroup that could make the algorithm easier to crack. 

Using the Python `cryptography` library, we can get a good $q$ value for us to use with the following code:

In [6]:
from cryptography.hazmat.primitives.asymmetric import dh

parameters = dh.generate_parameters(generator=2, key_size=2048)
param_numbers = parameters.parameter_numbers()
q = param_numbers.p
q

32270951546871796057874421669184811938446612398243265138793344475474149827083199982141595733088858697310218489831022161250115990653757650745922547676176590994566393727063050802667603182892435847266088197229598282834355979076245061851567928721385698565235689206998135911945709374573292524871364328569230226865353115303042536209972409490279824717398569537379392237625207749086553066629602984434276772718048816577775964998521887626982339410091532051211273960073261374720185498678737692878420211437029064163176224386166468098535035186666126335373712669173087969587795829952325157826577168368283215320785283238060701501903

In [7]:
g = param_numbers.g
g

2

Note that you can modify the generator and key_size to get a different sized $q$ and a different $g$ if you want.

Now that we have a safe and secure value for both $q$ and $g$, we can continue with our general explanation of Diffie-Hellman. 

Going back to the Alice and Bob example, let's have these two both pick their secret numbers randomly between 0 and $q$. Making sure these numbers are random is essential for security purposes, as if these numbers are not random, then they're not much of a secret. Let Alice's secret number be represented by $s_A$, and Bob's as $s_B$. 

Alice now calculates $$p_A=g^{s_A} \pmod q$$ He can do this using Python's secrets library to pick a value for $s_a$ and use the built-in function `pow(g, s_A, q)` to calculate $p_A$. We can calculate Bob's secret key using the same method: $$p_B=g^{s_B} \pmod q$$ These are their public keys, and they'll share them with each other. Now, they can get $k$, like they did in the previous example.  

Alice will raise Bob's public number to her secret key, rearranging it using our definition of $p_B$ above: $$ (p_B)^{s_A} = (g^{s_B})^{s_A} \pmod q $$ Bob does the opposite, raising Alice's public number to his secret: $$ (p_A)^{s_B}  = (g^{s_A})^{s_B} \pmod q $$

We can very clearly see from this that these two numbers are equal, as swapping the order of exponents does not matter. Exponents raised to exponents in this manner are simply multiplied, and multiplication is commutative. Thus, the final result is they both end up with: $$ k = (g^{s_B})^{s_A} = (g^{s_A})^{s_B} \pmod q $$ This is a number that they both know, but no one outside of them can calculate. 

Now that we have a safe and secure way to exchange a number, we can build a key exchange algorithm from this.

### The Key Exchange Algorithm

We will use Alice and Bob again for simplicity here. Suppose that Alice wants to send a message $m$="Hello World!" to Bob. 

In [8]:
message = "Hello World!"

In order to send this message securely using our algorithm, they both need to use the same $q$ and $g$, so Alice will send Bob the $q$ and $g$ she is planning on using over email. Keeping these secret is not vital to the security of the system, so it's okay to share them without encryption. Now that they both agree on $q$ and $g$, both Alice and Bob pick their secret numbers (s_A and s_B respectively in the code) between 0 and $q$ using python's `secrets` library in order to get numbers that are random enough to be used in cryptographic calculations:

In [9]:
s_A = secrets.randbelow(q)  
s_B = secrets.randbelow(q) 

Since we already have our $q$ and $g$, it's simple enough for both of them to compute their public keys: $$p_A=g^{s_A} \pmod q$$ $$ p_B=g^{s_B} \pmod q $$

In [10]:
p_A = pow(g, s_A, q)
p_B = pow(g, s_B, q)

Now, Alice and Bob will encode their keys. They will do this by first encoding their public keys as bytes. This is done because most python cryptography libraries expect input in bytes:

In [ ]:
p_A_bytes = int_to_bytes(p_A)
p_B_bytes = int_to_bytes(p_B)
p_A_bytes

b"\xa6\x96L)S\x15\t\xdf>\x1b\x199\xce\x05\xd5\xbbB\x05\xd3\xbcz\xc6\xd4\xbdOOJ\xac\xdbf\x84\x0c\t\xab\xe9\\\xc5P\xf8\x12\xd8\xe2u\x9dZI\xefm7\x15z\x87\x94u\xd6\xc6\xcc\xe2\x9f\xa2Y\xb6\xe0B\xfd\xbb\xc3\xf2p\x0f9-&F\x96)\x06\xd5\xec#\xf3\xf9\x11\x985\xa4rK}\xb1U\xb3\xaa+\xcb\xd0\x12\xed\xb5\x88\xa2W*#v,\xde\xc2r\xa6\x87\xcd\x11X\xc4\xe5\x1eq\x17\xe8\x05\xb3O\xd30Q.\x86\x08\xbe\xb0}\n\xbe5\xbf\xaa\xc3\xa1A\x8e\x87?<\x01pZuM'\x96[\x86\x004\xca\x8c\x02>HR\x03\x81\xa7\x1d\xf5\xbf5\xf4\xa0\x9f\xb8\x80\xc5J*\x88\xd5Nj#3\xdd\xeb\xa8\xad\xbf\x92\xf2\xcf\x9b\x07O\xa9\xc2\xaa_G\xbcd&*\x85\xd4\xb9)\xdfz\xdf\xea\xf7\\\xd2\x11\xa8\xdao\x03'\xab\xb1\x903\xb5\x19\x98\x15\xbc\x91\xae\xe9J\xaf\xc1\xd3sa#\xb2\x81@F\xc0U\x16\x0e\x0f\xbf\x01\x0f\x88\x86:1\x7f\xe2"

Within this byte representation, however, not all of the characters are ASCII characters, which can cause issues when transmitting data over email. Additionally, byte encoded messages are generally very long, so we convert the byte representation to Base64. This encoding contains all the same data, but in all ASCII characters, and in a more condensed format.

In [ ]:
p_A_b64 = bytes_to_Base64(p_A_bytes)
p_B_b64 = bytes_to_Base64(p_B_bytes)
p_A_b64

'ppZMKVMVCd8+Gxk5zgXVu0IF07x6xtS9T09KrNtmhAwJq+lcxVD4EtjidZ1aSe9tNxV6h5R11sbM4p+iWbbgQv27w/JwDzktJkaWKQbV7CPz+RGYNaRyS32xVbOqK8vQEu21iKJXKiN2LN7CcqaHzRFYxOUecRfoBbNP0zBRLoYIvrB9Cr41v6rDoUGOhz88AXBadU0nlluGADTKjAI+SFIDgacd9b819KCfuIDFSiqI1U5qIzPd66itv5Lyz5sHT6nCql9HvGQmKoXUuSnfet/q91zSEajabwMnq7GQM7UZmBW8ka7pSq/B03NhI7KBQEbAVRYOD78BD4iGOjF/4g=='

Now that they have their public keys, they share them with each other over email. Again, these keys are public, so sending them over email is not an issue. 

Once recieving each other's keys, both Alice and Bob undo all of the encoding, converting the keys back into integers from their Base64 representation:

In [ ]:
p_A_received_bytes = Base64_to_bytes(p_A_b64)
p_A_received = bytes_to_int(p_A_received_bytes)
p_B_received_bytes = Base64_to_bytes(p_B_b64)
p_B_received = bytes_to_int(p_B_received_bytes)

With each other's public keys, Alice and Bob can both calculate $k$, their shared secret: 

$$k_A = (p_B)^{s_A}  = (g^{s_B})^{s_A} \pmod q $$

$$k_B = (p_A)^{s_B}  = (g^{s_A})^{s_B} \pmod q $$

In [14]:
k_A = pow(p_B_received, s_A, q)
k_B = pow(p_A_received, s_B, q)

As we know from before, $k=k_A = k_B$, and thus they have their shared secret $k$:

In [15]:
k = k_A
assert k_A == k_B

They both then will convert this key $k$ into a byte representation, so that it can be actually used as an encryption key:

In [ ]:
key_bytes = int_to_bytes(k)

We do this so that we can use other cryptographic python functions such as `shake256` or others. 

### Encryption

We can now actually encrypt our message using this secret key. We will do this by first generating a `keystream`, which acts as a variable length key for our message, and it is the main component of our cipher, apart from the message itself. It utilizes a hash function to add another level of randomness and pattern removal from our message. 

In [ ]:
message_bytes = str_to_bytes(message)
keystream = generate_keystream(key_bytes, len(key_bytes))

With this, we now have a keystream that is secure and unpredictable, with no efficient way of deciphering the message from simply seeing this output. Now we can actually encrypt the message using our XOR cipher.

We use this function because we need an encryption that is reversible on the other end. If we were to simply feed our message into the hash function, no one could decrypt it, even with the secret key. By using the XOR function and applying it on our message and generated keystream, we are able to utilize the security offered by the hash function, while still being able to decrypt the message.

Using this function, we can apply the XOR to our `message` and `keystream` to give us a `bytes` object that has been encrypted:

In [ ]:
cipher_bytes = xor_bytes(message_bytes, keystream)

We now have the variable `cipher_bytes`, which is our encrypted message. Alice will then convert this to Base64 so that it can be transmitted safely:

In [ ]:
ciphertext_b64 = bytes_to_Base64(cipher_bytes)

Alice can now send this encrypted message over to Bob through email. 

Once Bob receives it, he can decode it with their shared secret. He starts by decoding the Base64 into the bytes that created it:

In [ ]:
received_cipher_bytes = Base64_to_bytes(ciphertext_b64)

He continues by calciulating the exact same keystream as Alice did:

In [ ]:
bob_keystream = generate_keystream(key_bytes,len(received_cipher_bytes))

He then applies the same XOR function to recover the original message:

In [ ]:
decrypted_bytes = xor_bytes(received_cipher_bytes, bob_keystream)

Bob can then finally convert those bytes back into plain text and read the message:

In [ ]:
decrypted_message = bytes_to_str(decrypted_bytes)
print("Bob decrypts and reads:", decrypted_message)

Bob decrypts and reads: Hello World!


That is the basis of this encryption scheme. Alice was able to send a message to Bob that no one without their shared secret could understand. Once they had both established `key_bytes` with each other, they each has a "key" that they could use to decrypt messages that the other send over. 

With this basic version of the algorithm in place, we can now talk about some of the potential issues with using this algorithm. 

### Potential Issue 1: Reusing the Keystream

In stream ciphers, the security of the message depends entirely on the randomness and uniqueness of the keystream used during encryption. If the same keystream is reused across multiple messages, attackers can exploit the deterministic nature of XOR to cancel out the keystream and reveal information about the original messages.

For example, consider two plaintext messages $M_1$ and $M_2$ encrypted with the same keystream $K$ derived from the same shared secret and the same nonce:

$$
C_1 = M_1 \oplus K, \quad C_2 = M_2 \oplus K
$$

An attacker who captures both ciphertexts can compute:

$$
C_1 \oplus C_2 = M_1 \oplus K \oplus M_2 \oplus K = M_1 \oplus M_2
$$

This result is the XOR of the original messages and leaks information about their relationship. If one plaintext is known or partially guessable (e.g. "Hello, my name is..."), the attacker can infer parts of the other.

This is why modern encryption systems always introduce some randomness or uniqueness per message—either with a nonce, counter, or initialization vector. In the full implementation provided in the scripts here, a nonce is used. It's a unique, one time value that is included with the data to add some randomness to every value sent.

Say you send these two messages, `m1` and `m2`, to someone else with out current algorithm:

In [ ]:
m1 = "Hello there"
m2 = "howdy partner"
m1_bytes = str_to_bytes(m1)
m2_bytes = str_to_bytes(m2)

ks = generate_keystream(key_bytes, max(len(m1), len(m2)))
cipher1 = xor_bytes(m1_bytes, ks)
cipher2 = xor_bytes(m2_bytes, ks)

Now, if an attacker were to intercept both of these messages, they could gleam information about `m1` or `m2` from `m1` $\oplus$ `m2` if they knew any part of either message:

In [ ]:
leak = xor_bytes(cipher1, cipher2)
print("Leaked (m1 ⊕ m2):", leak)

Leaked (m1 ⊕ m2): b' \n\x1b\x08\x16\x00\x04\t\x17\x06\x0b'


We can see that if the attacker knows `m1`, they can recover `m2`, or at least part of it:

In [ ]:
recovered_m2 = xor_bytes(leak, m1_bytes)
print("Recovered m2:", recovered_m2.decode('utf-8'))

Recovered m2: howdy partn


Now we will add in a nonce to the encoding process, and this makes it so that:
$$
C_1 \oplus C_2 = M_1 \oplus K_1 \oplus M_2 \oplus K_2 \neq M_1 \oplus M_2
$$

In [ ]:
nonce1 = secrets.token_bytes(16)
nonce2 = secrets.token_bytes(16)
ks1 = generate_keystream(key_bytes + nonce1, len(m1_bytes))
ks2 = generate_keystream(key_bytes + nonce2, len(m2_bytes))
c1  = xor_bytes(m1_bytes, ks1)
c2  = xor_bytes(m2_bytes, ks2)

leak2 = xor_bytes(c1, c2)
print("Leak with nonces    :", leak2)

Leak with nonces    : b"D\x1f[u\xf87\x9cy'\x9e\xd2"


Now, if the attacker tries to XOR together the messages, the output will be nonsense, even if he knows one of the messages already:

In [ ]:
leak = xor_bytes(cipher1, cipher2)
print("XOR of ciphertexts (as text):", leak.decode('utf-8', errors='replace'))

XOR of ciphertexts (as text):  
 	


### Potential Issue 2: Forged or Missing Message Authentication (No MAC Verification)

Encryption ensures confidentiality, but it does not protect message integrity or authenticity on its own. Without a mechanism to detect tampering, an attacker could flip bits in the ciphertext and alter the decrypted message in ways that are unpredictable but potentially harmful.

To address this, we use a Message Authentication Code (MAC)—specifically, `HMAC-SHA256`—which produces a short digest tied to both the ciphertext and the shared secret. The receiver verifies that the digest matches before trusting or decrypting the message.

If a message lacks a valid MAC or if the recipient fails to check it, the system becomes vulnerable to **forgery and tampering attacks**. In our notebook, we demonstrate that providing a forged MAC results in a verification failure, as expected. This shows that MAC verification is working—but also why omitting it would be disastrous.

In the code below, the attacker intercepted our cipher, and all they did was flip a single byte in it, which will change our message:

In [ ]:
tampered = bytearray(cipher1)
tampered[0] ^= 0x01
tampered = bytes(tampered)

decrypted_bad = bytes(c ^ ks[i] for i, c in enumerate(tampered))

print("Cipher1 (hex): ", cipher1.hex())
print("Tampered cipher (hex): ", tampered.hex())
print("Decrypted tampered text:", decrypted_bad.decode('utf-8', errors='replace'))

Cipher1 (hex):           aec0711813be413b9a6153
Tampered cipher (hex):   afc0711813be413b9a6153
Decrypted tampered text: Iello there


Implementing a MAC will add a way to tell you when your message is tampered with in transit:

In [ ]:
nonce = secrets.token_bytes(16)
blob = nonce + cipher1
original_mac = create_mac(key_bytes, blob)
print("Original MAC:", original_mac)

tampered = bytearray(cipher1)
tampered[0] ^= 0x01
tampered = bytes(tampered)
print("Tampered cipher (hex):", tampered.hex())

tampered_blob = nonce + tampered
new_mac = create_mac(key_bytes, tampered_blob)
print("New MAC:", new_mac)

if not hmac.compare_digest(original_mac, new_mac):
    print("MAC verification failed! Tampering detected—won’t decrypt.")
else:
    decrypted = bytes(c ^ ks[i] for i, c in enumerate(tampered))
    print("Decrypted:", decrypted.decode('utf-8', errors='replace'))

Original MAC: 331828d758d45d658aa0a236e2cb3b8b5e427f242edacb6778a0f2769b88dfe7
Tampered cipher (hex): afc0711813be413b9a6153
New MAC: 49a9d84b16e747f4b576bc7cef0f7ab02c8b7827e02d083f9db16f61ff2844cc
MAC verification failed! Tampering detected—won’t decrypt.


### Potential issue 3: Replay Attacks (No Freshness Guarantee)

Even if a message is encrypted and authenticated correctly, there is no mechanism in our current system to distinguish between an original message and a replayed one. If an attacker captures a valid message, they can resend it at a later time—potentially causing unintended behavior.

For example, if a message means "authorize transaction," a replay could execute the same command again. Because the MAC is still valid (it depends only on the message content and key), and because our code does not check timestamps, nonces, or sequence numbers, the message will be accepted as if it were new.

Defenses against replay attacks typically involve ensuring *freshness*—for instance, including a timestamp or random nonce in the message and rejecting duplicates or stale messages.


In [31]:
replayed = bytes(c ^ ks[i] for i, c in enumerate(cipher1))
print("Replayed message decrypted:", replayed.decode('utf-8'))

Replayed message decrypted: Hello there


A nonce is the easist solution to this, as it will add the "freshness" necessary so that each message is different, so our fix for potential issue #1 also works here. 

### Potential Issue 4: Lack of Forward Secrecy (Static Keys Reused)

Forward secrecy is the property that ensures that a compromise of long-term private keys does not allow an attacker to decrypt past communications. Our current implementation does **not** offer forward secrecy, because each party uses a long-term keypair, and once the Diffie-Hellman shared secret is established, it's used to encrypt multiple messages.

If an adversary ever obtains one party’s private key (say, through hacking or legal coercion), they can recompute the shared secrets for every past communication with the corresponding public keys. Since we don’t rotate or discard keys, **the attacker can retroactively decrypt all previous messages**.

In [ ]:
leaked_s_A = s_A
recovered_k = pow(p_B, leaked_s_A, q)
recovered_bytes = int_to_bytes(recovered_k)
recovered_ks = generate_keystream(recovered_bytes, len(m1_bytes))
recovered = bytes(c ^ recovered_ks[i] for i, c in enumerate(cipher1))
print("Recovered plaintext after key leak:", recovered.decode('utf-8'))

Recovered plaintext after key leak: Hello there


This has already been implemented in this algorithm, as both parties generate a new secret every time the algoritm is run. 